In [1]:
import ecdsa
import hashlib
import random
from gmpy2 import invert

In [2]:
G = ecdsa.SECP256k1.generator
N = ecdsa.SECP256k1.order

d = 2078

P1

In [3]:
p1_ans = 4 * G
print("G_x = {}\nG_y = {}".format(p1_ans.x(), p1_ans.y()))

G_x = 103388573995635080359749164254216598308788835304023601477803095234286494993683
G_y = 37057141145242123013015316630864329550140216928701153669873286428255828810018


P2

In [4]:
p2_ans = 5 * G
print("G_x = {}\nG_y = {}".format(p2_ans.x(), p2_ans.y()))

G_x = 21505829891763648114329055987619236494102133314575206970830385799158076338148
G_y = 98003708678762621233683240503080860129026887322874138805529884920309963580118


P3

In [5]:
Q = d * G
print("G_x = {}\nG_Y = {}".format(Q.x(), Q.y()))

G_x = 10558952611465060619920771719691208775840427052079655958727364975197137826262
G_Y = 99595754422124230268181950840807047095125375794236015674139284580993174427739


P4

In [6]:
def count_bits(integer):
    double = 0
    add = 0
    binary_representation = bin(integer)[2:]
    print("bin = {}".format(binary_representation))
    for bit in binary_representation[1:]:
        double += 1
        if (bit == '1'):
            add += 1
    return double, add
double, add = count_bits(d)
print("double = {}, add = {}".format(double, add))

bin = 100000011110
double = 11, add = 4


double = number of bits (excluding MSB) = 12 - 1 = 11 

add = number of 1 (excluding MSB) = 4

P5

<img src="P5.jpg">

P6

In [7]:
# this is the bitcoin transaction I used:
# https://www.blockchain.com/explorer/transactions/btc/a473c52545422e78f5ac9b46d97766d85a880cfa8acbc662ffd89a9bcae2a3ff

message = "a473c52545422e78f5ac9b46d97766d85a880cfa8acbc662ffd89a9bcae2a3ff"
# 1. Calculate e = HASH(m)
hash_func = hashlib.sha256()
hash_func.update(message.encode('utf-8'))
e = hash_func.hexdigest()
print("e = {}".format(e))

# 2. Let z be the Ln leftmost bits of e, where Ln is the bit length of the group order n.
Ln = N.bit_length()
Ln = min(Ln, len(message))
z = message[:Ln]
z = int(z, 16)
print("z = {}".format(z))

r = 0
s = 0
dA = 2078
while True:
    # 3. select a random integer k from [1, n-1]
    k = random.randint(1, N-1)
    # 4. Calculate the curve point (x1, y1) = k * G
    x1 = (k * G).x()
    y1 = (k * G).y()
    # 5. Calculate r = x1 mod n. If r = 0, go back to while loop
    r = x1 % N
    # 6. Calculate s = k^-1 (z + rdA) mod n. If s = 0, go back to while loop
    s = (invert(k, N) * (z + r * dA)) % N
    # 7. The signature is the pair (r, s)
    if r != 0 and s != 0:
        break
print("r = {}".format(r))
print("s = {}".format(s))

e = 5b2cb9133cc060148fd6bfab62f99a64315cb5703861cfc60ac948abbc1c6113
z = 74383855228947319945519443418357011162203109683400000045254379739972964099071
r = 39710263892406800111444519683998442045046138002057654872305930518776158620144
s = 70050608243560364735463861712822257272682854828353185822780992683343375419177


P7

In [10]:
# public key QA = dA * G
QA = dA * G

# 1. Verify that r and s are integers in [1, n-1]. If not, the signature is invalid
assert r >= 1 and r <= N-1 and s >= 1 and s <= N-1, "signature is invalid"
# 2. Calculate e = HASH(m), where HASH is the same function used in the signature generation
# 3. Let z be the Ln leftmost bits of e.
# 4. Calculate w = s^-1 mod n.
w = (invert(s, N)) % N
print("w = {}".format(w))
# 5. Calculate u1 = zw mod n and u2 = rw mod n.
u1 = (z * w) % N
u2 = (r * w) % N
print("u1 = {}".format(u1))
print("u2 = {}".format(u2))
# 6. Calculate the curve point (x1, y1) = u1 * G + u2 * QA.
x1 = (u1 * G + u2 * QA).x()
y1 = (u1 * G + u2 * QA).y()
print("x1 = {}".format(x1))
print("y1 = {}".format(y1))
# 7. The signature is valid if r = x1 (mod n), invalid otherwise.
if ((r % N) == (x1 % N)):
    print("Signature is valid")
else:
    print("Signature is invalid")

w = 95831231978288292503064569628370595003321793744457584320986667974298541646149
u1 = 18361929976077092449926493634644536486770132383154790200673392457774188298377
u2 = 44955536192030403449219372429176433705707763048168285286701526130643719807689
x1 = 39710263892406800111444519683998442045046138002057654872305930518776158620144
y1 = 68550685520807823803293341296170381561995055765667558701064178886341073333311
Signature is valid


P8

In [11]:
from sympy import symbols, Poly, GF

x = symbols('x')

c_lagrange = ((10 * (x - 2) * (x - 3) * invert(1 - 2, 10007) * invert(1 - 3, 10007)) + (20 * (x - 1) * (x - 3) * invert(2 - 1, 10007) * invert(2 - 3, 10007)) +
              (2078 * (x - 1) * (x - 2) * invert(3 - 1, 10007) * invert(3 - 2, 10007)))

p = Poly(c_lagrange, domain=GF(10007))
print(p)

Poly(1024*x**2 - 3062*x + 2048, x, modulus=10007)


<img src = "P8.jpg">